In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy, scipy as sklearn, librosa, urllib
import librosa.display
from IPython.display import Audio
import json 
import seaborn as sns
from sklearn.cluster import KMeans
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
import keras
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from os import path

from itertools import cycle
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc, silhouette_score,roc_auc_score, precision_recall_fscore_support
from tqdm import tqdm
import opensmile


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import regularizers
from joblib import dump, load

from tensorflow.keras.datasets import mnist


In [3]:
def return_features(file,pad):
    X, sample_rate = librosa.load(file)
    max_ = X.shape[0] / sample_rate
    # if max_ < pad:
    #     length = (pad * sample_rate) -  (X.shape[0] / sample_rate)
    #     X = librosa.util.pad_center(X, size = length, mode = 'constant')
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
    )
    y = smile.process_signal(X,sample_rate)
    #  smile.process_file(file, end=3)
    return y
def get_max_min(files):
    min_, max_ = 100, 0
    for file in files:
        sound_file, samplerate = librosa.load(file)
        t = sound_file.shape[0] / samplerate
        if t < min_:
            min_ = t
        if t >= max_:
            max_ = t

    return np.round(max_,2) + 0.01, min_

In [5]:
#add white noise to the original signal
def noise_addition(data,noise_percentage_factor=0.035):
    noise = np.random.normal(0, data.std(), data.size)
    augmented_data = data + noise * noise_percentage_factor
    return augmented_data

#lower the pitch of the original signal
def pitch_scaling(data, sr, num_semitones=-2):
    return librosa.effects.pitch_shift(y = data,sr = sr,n_steps = num_semitones)

#increase the pitch of the original signal
def pitch_scaling2(data, sr, num_semitones=2):
    return librosa.effects.pitch_shift(y = data,sr = sr,n_steps = num_semitones)



In [5]:
data_df = pd.read_csv("DEMoS/data.csv")

### CLEAR

In [ ]:
train_data = pd.DataFrame(columns=['filename', 'features', 'label'])
max, min = get_max_min('DEMoS/'+data_df.file_name)
filenames= data_df.file_name
labels= data_df.label

tot = range(list(data_df.index)[-1])



for index,file in tqdm(zip(tot,filenames)):
    train_data.loc[index] = [file, return_features('DEMoS/'+file,max), labels[index]]


#### PITCH ONLY

In [2]:
def return_features(file,pad):
    X, sample_rate = librosa.load(file)
    max_ = X.shape[0] / sample_rate
    # if max_ < pad:
    #     length = (pad * sample_rate) -  (X.shape[0] / sample_rate)
    #     X = librosa.util.pad_center(X, size = length, mode = 'constant')
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
    )

    X = pitch_scaling(X, sample_rate)
    y = smile.process_signal(X,sample_rate)
    #  smile.process_file(file, end=3)
    return y

def return_features2(file,pad):
    X, sample_rate = librosa.load(file)
    max_ = X.shape[0] / sample_rate
    # if max_ < pad:
    #     length = (pad * sample_rate) -  (X.shape[0] / sample_rate)
    #     X = librosa.util.pad_center(X, size = length, mode = 'constant')
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
    )

    X = pitch_scaling2(X, sample_rate)
    y = smile.process_signal(X,sample_rate)
    #  smile.process_file(file, end=3)
    return y

def get_max_min(files):
    min_, max_ = 100, 0
    for file in files:
        sound_file, samplerate = librosa.load(file)
        t = sound_file.shape[0] / samplerate
        if t < min_:
            min_ = t
        if t >= max_:
            max_ = t

    return np.round(max_,2) + 0.01, min_

In [3]:
data_df = pd.read_csv("EMOVO_dataset/data.csv")

In [6]:
train_data = pd.DataFrame(columns=['filename', 'features', 'label'])
max, min = get_max_min('EMOVO_dataset/'+data_df.file_name)
filenames= pd.concat([data_df.file_name]*2, ignore_index=True) 
labels= pd.concat([data_df.label]*2, ignore_index=True) 
features = []
tot = range(list(data_df.index)[-1]*2)
tot_m = (list(data_df.index)[-1]*2)

for index,file in tqdm(zip(tot,filenames)):
    if index <= tot_m//2:
        train_data.loc[index] = [file, return_features('EMOVO_dataset/'+file,max), labels[index]]
    else:
        train_data.loc[index] = [file, return_features2('EMOVO_dataset/'+file,max), labels[index]]

1174it [01:38, 11.93it/s]


In [7]:
train_data.to_pickle("pitch_functional_emovo.pkl")